# Extracting simplified geometry from 3d microscopy volumes

This notebook illustrates how to derive a weighted undirected graph from a 3d dense volume.
The undirected graph provides a sort of simplified description of the geometry of isosurfaces of the volume.

First get volume data:

In [ ]:
%ls ../../misc_data

In [ ]:
fn = "../../misc_data/trachea_raw_8p.tif"

In [ ]:
from mouse_embryo_labeller import tools
import numpy as np
from scipy.ndimage import gaussian_filter

In [ ]:
# Load the full resolution raw data.
V = tools.load_tiff_array(fn)

# Smooth the volume to eliminate some noise
V = gaussian_filter(V, sigma=4)

# Subsample the volume to a smaller size to make it tractible.
stride = 4
Vol = V[:, ::stride, ::stride]

# Flip axes for presentation.
Vol = np.swapaxes(Vol, 0, 2)
Vol.shape

In [ ]:
# Slice out the section of the volume of interest
Vsliced = Vol[29:105, 0:128, 0:21]
Vsliced.shape

In [ ]:
# View the volume and optionally adjust the view to select the isosurface of interest.
from feedWebGL2 import volume

volume.widen_notebook()
W = volume.Volume32()

W

In [ ]:
# The 'tetrahedra' method supports capturing triangle and normal geometry from the volume
W.load_3d_numpy_array(Vsliced, threshold=8.1, axis_length=False, method="tetrahedra", dk=dict(x=0, y=0, z=0.2))

x = W.build(1500)

In [ ]:
# Get the isosurface triangles from the volume display geometry.
triangles = W.triangles_and_normals(just_triangles=True)

triangles.shape

In [ ]:
# Initialize the topology analysis tool with the isosurface triangle geometry
# Initialization takes some time.
from mouse_embryo_labeller import  topoFinder

T = topoFinder.TopologyFinder(triangles)
len(T.vertex_to_edges)

In [ ]:
# Collapse edges until all triangles have been eliminated.
# This step takes a while.
count = 0
while T.triangle_to_edges:
    count += 1
    if count % 10000 == 1:
        print(count, len(T.vertex_to_edges))
    T.optimized_collapse_edge(redraw=False)

In [ ]:
print(count, len(T.vertex_to_edges))

In [ ]:
# View the simplified geometry (2D view)
T.doodle()

In [ ]:
# Extract the derived vertices, weights, and edges for downstream processing.
(rescaled_vertices, rescaled_weights, edge_indices) = T.vertices_weights_and_edge_indices()

In [ ]:
# Examine the vertices:
print(rescaled_vertices.shape, rescaled_vertices.min(axis=0), rescaled_vertices.max(axis=0))
rescaled_vertices[:10]

In [ ]:
# Examine the vertex weights:
print(rescaled_weights.shape, rescaled_weights.min(), rescaled_weights.max())
rescaled_weights[:10]

In [ ]:
# Examine the undirected edges:
print(edge_indices.shape)
edge_indices[:10]